In [38]:
pip install yfinance

First, let's import the necessary libraries: `yfinance` for fetching stock data and `pandas` for data manipulation.

In [39]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the stock ticker and the end date (today)
ticker = 'SPY' # You can change this to any stock ticker
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365) # Approximately 5 years ago

Now, let's download the historical daily stock data for the specified period.

In [40]:
df = yf.download(ticker, start=start_date, end=end_date)

# Display the first few rows of the raw data
print(f"Downloaded data for {ticker}:")
display(df.head())

/tmp/ipython-input-2650319210.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

Downloaded data for SPY:


Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2020-12-28,347.411865,347.803909,346.385036,347.010449,39000400
2020-12-29,346.749115,349.120153,346.161020,348.942790,53680500
2020-12-30,347.243866,348.280040,346.851822,347.570588,49455300
2020-12-31,349.008148,349.736259,346.534442,347.047842,78520700
2021-01-04,344.256836,350.473793,340.550933,350.343093,110210800


Next, we'll calculate the 5-day, 50-day, and 200-day Simple Moving Averages (SMAs) based on the 'Close' price.

In [41]:
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_50'] = df['Close'].rolling(window=50).mean()
df['SMA_200'] = df['Close'].rolling(window=200).mean()

# Display the DataFrame with the new moving average columns
print("DataFrame with moving averages:")
display(df.tail()) # Display the tail as MAs will be NaN for the initial rows

DataFrame with moving averages:


Price,Close,High,Low,Open,Volume,SMA_5,SMA_50,SMA_200
Ticker,SPY,SPY,SPY,SPY,SPY,,,
Date,,,,,,,,
2025-12-18,674.476929,678.734368,672.911608,675.603604,108650100,675.848926,672.912211,618.330229
2025-12-19,680.590027,681.090027,676.469971,676.590027,103599500,676.016650,673.140359,618.902866
2025-12-22,684.830017,685.359985,680.590027,683.940002,69556700,677.237769,673.815038,619.480839
2025-12-23,687.960022,688.200012,683.869995,683.919983,64840000,679.455786,674.352507,620.150272
2025-12-24,690.380005,690.830017,687.799988,687.950012,39445600,683.647400,674.954529,620.854834


Finally, let's clean up the DataFrame to only include the requested columns and display the first 5 rows.

In [42]:
final_df = df[['Open', 'High', 'Low', 'Close', 'SMA_5', 'SMA_50', 'SMA_200', 'Volume']].copy()
print("Final DataFrame with requested columns:")
display(final_df.head())

Final DataFrame with requested columns:


Price,Open,High,Low,Close,SMA_5,SMA_50,SMA_200,Volume
Ticker,SPY,SPY,SPY,SPY,,,,SPY
Date,,,,,,,,
2020-12-28,347.010449,347.803909,346.385036,347.411865,NaN,NaN,NaN,39000400
2020-12-29,348.942790,349.120153,346.161020,346.749115,NaN,NaN,NaN,53680500
2020-12-30,347.570588,348.280040,346.851822,347.243866,NaN,NaN,NaN,49455300
2020-12-31,347.047842,349.736259,346.534442,349.008148,NaN,NaN,NaN,78520700
2021-01-04,350.343093,350.473793,340.550933,344.256836,346.933966,NaN,NaN,110210800


First, let's calculate the daily logarithmic returns from the 'Close' price.

In [43]:
import numpy as np

# Ensure Close prices are numeric and fill any potential NaNs before calculating returns
# This is a common pre-processing step for financial time series
cleaned_close = df['Close'].ffill().bfill() # Forward fill then backward fill to handle leading NaNs

# Calculate daily logarithmic returns
# Use the cleaned_close series for calculation
df['Log_Return'] = np.log(cleaned_close / cleaned_close.shift(1))

print("DataFrame with Log Returns:")
display(df.head())

DataFrame with Log Returns:


Price,Close,High,Low,Open,Volume,SMA_5,SMA_50,SMA_200,Log_Return
Ticker,SPY,SPY,SPY,SPY,SPY,,,,
Date,,,,,,,,,
2020-12-28,347.411865,347.803909,346.385036,347.010449,39000400,NaN,NaN,NaN,NaN
2020-12-29,346.749115,349.120153,346.161020,348.942790,53680500,NaN,NaN,NaN,-0.001910
2020-12-30,347.243866,348.280040,346.851822,347.570588,49455300,NaN,NaN,NaN,0.001426
2020-12-31,349.008148,349.736259,346.534442,347.047842,78520700,NaN,NaN,NaN,0.005068
2021-01-04,344.256836,350.473793,340.550933,350.343093,110210800,346.933966,NaN,NaN,-0.013707


Next, we'll calculate the annualized historical volatility and then the daily Expected Move. We'll use 252 trading days for annualization.

In [44]:
# Calculate annualized historical volatility
daily_volatility = df['Log_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

# Calculate daily Expected Move (as 1 standard deviation move)
df[('Expected_Move', 'SPY')] = df[('Close', 'SPY')] * annualized_volatility * np.sqrt(1/252)

# Add Expected_Move to final_df, ensuring MultiIndex compatibility
final_df[('Expected_Move', 'SPY')] = df[('Expected_Move', 'SPY')]

print("Final DataFrame with Expected Move:")
display(final_df.tail())

Final DataFrame with Expected Move:


Price,Open,High,Low,Close,SMA_5,SMA_50,SMA_200,Volume,Expected_Move
Ticker,SPY,SPY,SPY,SPY,,,,SPY,SPY
Date,,,,,,,,,
2025-12-18,675.603604,678.734368,672.911608,674.476929,675.848926,672.912211,618.330229,108650100,7.262269
2025-12-19,676.590027,681.090027,676.469971,680.590027,676.016650,673.140359,618.902866,103599500,7.328090
2025-12-22,683.940002,685.359985,680.590027,684.830017,677.237769,673.815038,619.480839,69556700,7.373743
2025-12-23,683.919983,688.200012,683.869995,687.960022,679.455786,674.352507,620.150272,64840000,7.407445
2025-12-24,687.950012,690.830017,687.799988,690.380005,683.647400,674.954529,620.854834,39445600,7.433501


Let's add a new column to `final_df` that represents the difference between the 'Open' and 'Close' prices.

In [50]:
final_df[('Open_Close_Diff', 'SPY')] = final_df[('Close', 'SPY')] - final_df[('Open', 'SPY')]

print("Final DataFrame with 'Open_Close_Diff' column:")
display(final_df.tail())

Final DataFrame with 'Open_Close_Diff' column:


Price,Open,High,Low,Close,SMA_5,SMA_50,SMA_200,Volume,Expected_Move,Open_Close_Diff,Daily_Range
Ticker,SPY,SPY,SPY,SPY,,,,SPY,SPY,SPY,SPY
Date,,,,,,,,,,,
2025-12-18,675.603604,678.734368,672.911608,674.476929,675.848926,672.912211,618.330229,108650100,7.262269,-1.126676,5.822760
2025-12-19,676.590027,681.090027,676.469971,680.590027,676.016650,673.140359,618.902866,103599500,7.328090,4.000000,4.620056
2025-12-22,683.940002,685.359985,680.590027,684.830017,677.237769,673.815038,619.480839,69556700,7.373743,0.890015,4.769958
2025-12-23,683.919983,688.200012,683.869995,687.960022,679.455786,674.352507,620.150272,64840000,7.407445,4.040039,4.330017
2025-12-24,687.950012,690.830017,687.799988,690.380005,683.647400,674.954529,620.854834,39445600,7.433501,2.429993,3.030029


Now, let's add a 'Daily_Range' column, calculated as the difference between the 'High' and 'Low' prices.

In [51]:
final_df[('Daily_Range', 'SPY')] = final_df[('High', 'SPY')] - final_df[('Low', 'SPY')]

print("Final DataFrame with 'Daily_Range' column:")
display(final_df.tail())

Final DataFrame with 'Daily_Range' column:


Price,Open,High,Low,Close,SMA_5,SMA_50,SMA_200,Volume,Expected_Move,Open_Close_Diff,Daily_Range
Ticker,SPY,SPY,SPY,SPY,,,,SPY,SPY,SPY,SPY
Date,,,,,,,,,,,
2025-12-18,675.603604,678.734368,672.911608,674.476929,675.848926,672.912211,618.330229,108650100,7.262269,-1.126676,5.822760
2025-12-19,676.590027,681.090027,676.469971,680.590027,676.016650,673.140359,618.902866,103599500,7.328090,4.000000,4.620056
2025-12-22,683.940002,685.359985,680.590027,684.830017,677.237769,673.815038,619.480839,69556700,7.373743,0.890015,4.769958
2025-12-23,683.919983,688.200012,683.869995,687.960022,679.455786,674.352507,620.150272,64840000,7.407445,4.040039,4.330017
2025-12-24,687.950012,690.830017,687.799988,690.380005,683.647400,674.954529,620.854834,39445600,7.433501,2.429993,3.030029


Let's add a new column to `final_df` that represents the difference between the 'Open' and 'Close' prices.

In [54]:
final_df[('Open_Close_Diff', 'SPY')] = final_df[('Close', 'SPY')] - final_df[('Open', 'SPY')]

print("Final DataFrame with 'Open_Close_Diff' column:")
display(final_df.tail(100))

Final DataFrame with 'Open_Close_Diff' column:


Price,Open,High,Low,Close,SMA_5,SMA_50,SMA_200,Volume,Expected_Move,Open_Close_Diff,Daily_Range
Ticker,SPY,SPY,SPY,SPY,,,,SPY,SPY,SPY,SPY
Date,,,,,,,,,,,
2025-08-05,628.186955,629.002286,623.464044,624.388733,625.890161,609.025946,582.754134,68051400,6.722956,-3.798222,5.538242
2025-08-06,625.462617,629.827596,624.547881,629.171387,625.556079,610.127169,583.031937,64357500,6.774452,3.708770,5.279715
2025-08-07,632.611584,633.347354,625.522241,628.644348,625.589868,610.979130,583.306859,74205800,6.768778,-3.967236,7.825113
2025-08-08,630.444062,634.013616,630.125880,633.546265,628.664246,611.996938,583.595257,64051600,6.821558,3.102202,3.887736
2025-08-11,633.824714,635.306207,631.040633,632.293457,629.608838,612.943693,583.882120,58742300,6.808068,-1.531257,4.265574
...,...,...,...,...,...,...,...,...,...,...,...
2025-12-18,675.603604,678.734368,672.911608,674.476929,675.848926,672.912211,618.330229,108650100,7.262269,-1.126676,5.822760
2025-12-19,676.590027,681.090027,676.469971,680.590027,676.016650,673.140359,618.902866,103599500,7.328090,4.000000,4.620056


In [53]:
print("Correlation Matrix:")
display(correlation_matrix)

Correlation Matrix:


,Price,Open,High,Low,Close,SMA_5,SMA_50,SMA_200,Volume
,Ticker,SPY,SPY,SPY,SPY,,,,SPY
Price,Ticker,,,,,,,,
Open,SPY,1.000000,0.999571,0.999535,0.999055,0.998947,0.984885,0.948682,-0.288575
High,SPY,0.999571,1.000000,0.999284,0.999534,0.999107,0.986764,0.951989,-0.278557
Low,SPY,0.999535,0.999284,1.000000,0.999532,0.998500,0.982695,0.945493,-0.302833
Close,SPY,0.999055,0.999534,0.999532,1.000000,0.998481,0.984494,0.948767,-0.292395
SMA_5,,0.998947,0.999107,0.998500,0.998481,1.000000,0.987135,0.951165,-0.278102
SMA_50,,0.984885,0.986764,0.982695,0.984494,0.987135,1.000000,0.970686,-0.228575
SMA_200,,0.948682,0.951989,0.945493,0.948767,0.951165,0.970686,1.000000,-0.232674
Volume,SPY,-0.288575,-0.278557,-0.302833,-0.292395,-0.278102,-0.228575,-0.232674,1.000000
